# ADA 2018 - Homework 3



## Undestanding the StackOverflow community


Deadline: Nov 7th 2018, 23:59:59

Submission link: Check channel homework-3-public

StackOverflow is the most popular programming-related Q&A website. It serves as a platform for users to ask and answer questions and to vote questions and answers up or down. Users of StackOverflow can earn reputation points and "badges"; for example, a person is awarded 10 reputation points for receiving an "up" vote on an answer given to a question, and 5 points for the "up" vote on a question asked. Also, users receive badges for their valued contributions, which represents a kind of gamification of the traditional Q&A site. 

[Learn more about StackOverflow on Wikipedia](https://en.wikipedia.org/wiki/Stack_Overflow)

----

Dataset link:

https://drive.google.com/open?id=1POlGjqzw9v_pZ_bUnXGihOgk45kbvNjB

http://iccluster053.iccluster.epfl.ch/Posts.json.zip (mirror 1)

https://iloveadatas.com/datasets/Posts.json.zip (mirror 2)

Dataset description:

* **Id**: Id of the post
* **CreationDate**: Creation date of the post (String format)
* **PostTypeId**: Type of post (Question = 1, Answer = 2)
* **ParentId**: The id of the question. Only present if PostTypeId = 2
* **Score**: Points assigned by the users
* **Tags**: Tags of the question. Only present if PostTypeId = 1
* **Title**: Only present if PostTypeId = 1
* **ViewCount**: Only present if PostTypeId = 1

The dataset format is JSON. Here are examples of a question and an answer:

Question:
```json
{
    "Id": 10130734,
    "CreationDate": "2012-04-12T19:51:25.793+02:00",
    "PostTypeId": 1,
    "Score": 4,
    "Tags": "<python><pandas>",
    "Title": "Best way to insert a new value",
    "ViewCount": 3803
}
```

Answer:
```json
{  
   "CreationDate":"2010-10-26T03:19:05.063+02:00",
   "Id":4020440,
   "ParentId":4020214,
   "PostTypeId":2,
   "Score":1
}
```

----
Useful resources:

**Spark SQL, DataFrames and Datasets Guide**

https://spark.apache.org/docs/latest/sql-programming-guide.html

**Database schema documentation for the public data dump**

https://meta.stackexchange.com/questions/2677/database-schema-documentation-for-the-public-data-dump-and-sede

----

**Note:** Use Spark where possible. Some computations can take more than 10 minutes on a common notebook. Consider to save partial results on disk.

In [8]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp
from pyspark.sql import *
from pyspark.sql.functions import col
%matplotlib inline

spark = SparkSession.builder.getOrCreate()


### Task A: Convert the dataset to a more convenient format
As a warm-up task (and to avoid to warm up your laptop too much), load the dataset into a Spark dataframe, show the content, and save it in the _Parquet_ format. Use this step to convert the fields to a more convenient form.

Answer the following questions:

1. How many questions have been asked on StackOverflow?
2. How many answers have been given?
3. What is the percentage of questions with a score of 0?

**Hint:** The next tasks involve a time difference. Consider storing time in numeric format.

In [7]:
# Add your code and description here
posts = spark.read.json("Posts.json.zip")
posts

DataFrame[_corrupt_record: string]

In [12]:
print("Columns in the DataFrame:")
for column in posts.columns:
    print(column)

Columns in the DataFrame:
_corrupt_record


In [9]:
# 1. How many questions have been asked on StackOverflow?
questions_count = posts.filter(col("PostTypeId") == 1).count()

# 2. How many answers have been given?
answers_count = posts.filter(col("PostTypeId") == 2).count()

# 3. What is the percentage of questions with a score of 0?
total_questions = posts.filter(col("PostTypeId") == 1).count()
zero_score_questions = posts.filter((col("PostTypeId") == 1) & (col("Score") == 0)).count()

percentage_zero_score_questions = (zero_score_questions / total_questions) * 100

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `PostTypeId` cannot be resolved. Did you mean one of the following? [`_corrupt_record`].;
'Filter ('PostTypeId = 1)
+- Relation [_corrupt_record#34] json


**Hint:** Load the dataset from the Parquet file for the next tasks.

### Task B: What are the 10 most popular tags?

What are the most popular tags in StackOverflow? Use Spark to extract the information you need, and answer the following questions with Pandas and Matplotlib (or Seaborn):

1. What is the proportion of tags that appear in fewer than 100 questions?
2. Plot the distribution of the tag counts using an appropriate representation.
3. Plot a bar chart with the number of questions for the 10 most popular tags.

For each task describe your findings briefly.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Explode the Tags column to get individual tags
tags_df = posts.select("Tags").withColumn("Tag", explode(split("Tags", "><")))

# Remove angle brackets from tags
tags_df = tags_df.withColumn("Tag", tags_df["Tag"].substr(2, len(tags_df["Tag"]) - 2))

# Group by tags and count the occurrences
tag_counts = tags_df.groupBy("Tag").count()

# Cache the DataFrame for better performance
tag_counts.cache()

# Show the top 10 most popular tags
top_tags = tag_counts.orderBy("count", ascending=False).limit(10)
top_tags.show()

# Stop the Spark session
spark.stop()


In [3]:
# Add your code and description here
top_tags_pd = top_tags.toPandas()

# 1. What is the proportion of tags that appear in fewer than 100 questions?
less_than_100 = tag_counts.filter("count < 100").count()
total_tags = tag_counts.count()
proportion_less_than_100 = less_than_100 / total_tags

print("Proportion of tags appearing in fewer than 100 questions:", proportion_less_than_100)


In [ ]:
import matplotlib.pyplot as plt

# 2. Plot the distribution of tag counts using a histogram
plt.figure(figsize=(10, 6))
plt.hist(tag_counts.select("count").rdd.flatMap(lambda x: x).collect(), bins=50, color="skyblue", edgecolor="black")
plt.title("Distribution of Tag Counts")
plt.xlabel("Number of Questions")
plt.ylabel("Frequency")
plt.show()


In [ ]:
# 3. Plot a bar chart with the number of questions for the 10 most popular tags
plt.figure(figsize=(12, 8))
plt.bar(top_tags_pd["Tag"], top_tags_pd["count"], color="orange")
plt.title("Top 10 Most Popular Tags")
plt.xlabel("Tag")
plt.ylabel("Number of Questions")
plt.xticks(rotation=45, ha="right")
plt.show()

### Task C: View-score relation

We want to investigate the correlation between the view count and the score of questions.

1. Get the view count and score of the questions with tag ```random-effects``` and visualize the relation between these two variables using an appropriate plot.
2. Are these two variables correlated? Use the Pearson coefficient to validate your hypothesis. Discuss your findings in detail.

**Hint:** Inspect the data visually before drawing your conclusions.

In [4]:
# Add your code and description here

import seaborn as sns 

# Filter questions with the tag 'random-effects'
random_effects_questions = posts.filter((col("PostTypeId") == 1) & (col("Tags").contains("<random-effects>")))

# Select only the relevant columns (ViewCount and Score)
view_score_data = random_effects_questions.select("ViewCount", "Score").na.drop()

# Convert to Pandas for visualization
view_score_pd = view_score_data.toPandas()

# 1. Visualize the relation between ViewCount and Score using a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x="ViewCount", y="Score", data=view_score_pd)
plt.title("View Count vs. Score for Questions with Tag 'random-effects'")
plt.xlabel("View Count")
plt.ylabel("Score")
plt.show()

# 2. Calculate the Pearson correlation coefficient
correlation_coefficient = view_score_pd["ViewCount"].corr(view_score_pd["Score"], method="pearson")
print("Pearson Correlation Coefficient:", correlation_coefficient)


### Task D: What are the tags with the fastest first answer?

What are the tags that have the fastest response time from the community? We define the response time as the difference in seconds between the timestamps of the question and of the first answer received.

1. Get the response time for the first answer of the questions with the tags ```python``` and ```java```.
2. Plot the two distributions in an appropriate format. What do you observe? Describe your findings and discuss the following distribution properties: mean, median, standard deviation.
3. We believe that the response time is lower for questions related to Python (compare to Java). Contradict or confirm this assumption by estimating the proper statistic with bootstrapping. Visualize the 95% confidence intervals with box plots and describe your findings.
3. Repeat the first analysis (D1) by using the proper statistic to measure the response time for the tags that appear at least 5000 times. Plot the distribution of the 10 tags with the fastest response time.


In [5]:
# Add your code and description here
from scipy.stats import ttest_ind
from pyspark.sql.functions import col, when, min, unix_timestamp

# Filter questions with the tags 'python' and 'java'
selected_tags = ['python', 'java']
selected_questions = posts.filter((col("PostTypeId") == 1) & col("Tags").contains("|".join(selected_tags)))

# Calculate the response time for the first answer
response_time_data = selected_questions.select(
    "Id", "CreationDate",
    min(when(col("PostTypeId") == 2, col("CreationDate"))).over(Window.partitionBy("Id")).alias("FirstAnswerDate")
)

response_time_data = response_time_data.withColumn(
    "ResponseTime",
    (unix_timestamp("FirstAnswerDate") - unix_timestamp("CreationDate")).cast("int")
).na.drop()

# Convert to Pandas for visualization
response_time_pd = response_time_data.toPandas()


In [ ]:

# 2. Plot the distributions for Python and Java
plt.figure(figsize=(12, 8))
sns.histplot(response_time_pd, x="ResponseTime", hue="Tags", element="step", common_norm=False, stat="density", kde=True)
plt.title("Distribution of Response Time for Python and Java Questions")
plt.xlabel("Response Time (seconds)")
plt.ylabel("Density")
plt.show()

# Calculate mean, median, and standard deviation
mean_python = response_time_pd[response_time_pd['Tags'] == 'python']['ResponseTime'].mean()
median_python = response_time_pd[response_time_pd['Tags'] == 'python']['ResponseTime'].median()
std_python = response_time_pd[response_time_pd['Tags'] == 'python']['ResponseTime'].std()

mean_java = response_time_pd[response_time_pd['Tags'] == 'java']['ResponseTime'].mean()
median_java = response_time_pd[response_time_pd['Tags'] == 'java']['ResponseTime'].median()
std_java = response_time_pd[response_time_pd['Tags'] == 'java']['ResponseTime'].std()

print("Statistics for Python:")
print("Mean:", mean_python)
print("Median:", median_python)
print("Standard Deviation:", std_python)

print("\nStatistics for Java:")
print("Mean:", mean_java)
print("Median:", median_java)
print("Standard Deviation:", std_java)


In [ ]:
# 3. Use bootstrapping to estimate the 95% confidence intervals and visualize with box plots
def bootstrap_ci(data, n_bootstrap=1000, alpha=0.05):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(bootstrap_sample))

    lower_ci = np.percentile(bootstrapped_means, alpha / 2 * 100)
    upper_ci = np.percentile(bootstrapped_means, (1 - alpha / 2) * 100)

    return lower_ci, upper_ci

# Bootstrap and calculate confidence intervals for Python and Java
python_ci = bootstrap_ci(response_time_pd[response_time_pd['Tags'] == 'python']['ResponseTime'])
java_ci = bootstrap_ci(response_time_pd[response_time_pd['Tags'] == 'java']['ResponseTime'])

# Visualize the 95% confidence intervals with box plots
plt.figure(figsize=(10, 6))
sns.boxplot(x="Tags", y="ResponseTime", data=response_time_pd)
plt.title("Response Time Distribution with 95% Confidence Intervals")
plt.xlabel("Tags")
plt.ylabel("Response Time (seconds)")
plt.ylim(0, 1000)  # Adjust ylim for better visualization
plt.show()

print("\n95% Confidence Intervals:")
print("Python:", python_ci)
print("Java:", java_ci)

In [ ]:
# 4. Repeat the first analysis (D1) for the tags that appear at least 5000 times
tag_counts = posts.filter(col("PostTypeId") == 1).groupBy("Tags").count().filter("count >= 5000")

# Extract the tags with at least 5000 occurrences
popular_tags = [row["Tags"] for row in tag_counts.collect()]

# Filter questions with popular tags
popular_tags_questions = posts.filter((col("PostTypeId") == 1) & col("Tags").isin(popular_tags))

# Calculate the response time for the first answer
response_time_popular_tags = popular_tags_questions.select(
    "Id", "CreationDate",
    min(when(col("PostTypeId") == 2, col("CreationDate"))).over(Window.partitionBy("Id")).alias("FirstAnswerDate")
)

response_time_popular_tags = response_time_popular_tags.withColumn(
    "ResponseTime",
    (unix_timestamp("FirstAnswerDate") - unix_timestamp("CreationDate")).cast("int")
).na.drop()

# Convert to Pandas for visualization
response_time_popular_tags_pd = response_time_popular_tags.toPandas()

# Select the top 10 tags with the fastest response time
top_10_tags = response_time_popular_tags_pd.groupby("Tags").mean().sort_values(by="ResponseTime").head(10)

# Plot the distribution of the 10 tags with the fastest response time
plt.figure(figsize=(12, 8))
sns.barplot(x=top_10_tags.index, y=top_10_tags["ResponseTime"], palette="viridis")
plt.title("Top 10 Tags with the Fastest Response Time")
plt.xlabel("Tag")
plt.ylabel("Average Response Time (seconds)")
plt.xticks(rotation=45, ha="right")
plt.show()

### Task E: What's up with PySpark?
The number of questions asked regarding a specific topic reflect the public’s interest on it. We are interested on the popularity of PySpark. Compute and plot the number of questions with the ```pyspark``` tag for 30-day time intervals. Do you notice any trend over time? Is there any correlation between time and number of questions?


In [ ]:
# Add your code and description here

# Filter questions with the 'pyspark' tag
pyspark_questions = posts.filter((col("PostTypeId") == 1) & col("Tags").contains("<pyspark>"))

# Extract the creation date and convert it to date format
pyspark_questions = pyspark_questions.withColumn("CreationDate", to_date(col("CreationDate")))

# Group by 30-day intervals and count the number of questions
questions_per_interval = pyspark_questions.groupBy(window("CreationDate", "30 days")).agg(count("*").alias("NumQuestions"))

# Convert to Pandas for visualization
questions_per_interval_pd = questions_per_interval.toPandas()

# Plot the number of questions with the 'pyspark' tag over time
plt.figure(figsize=(12, 8))
plt.plot(questions_per_interval_pd["window.start"], questions_per_interval_pd["NumQuestions"], marker='o', linestyle='-')
plt.title("Number of Questions with 'pyspark' Tag Over Time (30-day Intervals)")
plt.xlabel("Time")
plt.ylabel("Number of Questions")
plt.xticks(rotation=45, ha="right")
plt.show()

In [1]:
# stop the spark session

spark.stop()

NameError: name 'spark' is not defined